# Hello Loopy: Computing a Rank-One Matrix

## Setup Code

In [2]:
import numpy as np
import pyopencl as cl
import pyopencl.array
import pyopencl.clrandom
import loopy as lp

In [3]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

In [4]:
n = 1024
a = cl.clrandom.rand(queue, n, dtype=np.float32)
b = cl.clrandom.rand(queue, n, dtype=np.float32)

## The Initial Kernel

In [5]:
knl = lp.make_kernel(
    "{[i,j]: 0<=i,j<n}",
    "c[i, j] = a[i]*b[j]")

In [6]:
knl = lp.set_options(knl, write_cl=True)
evt, (mat,) = knl(queue, a=a, b=b)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(__global float const *restrict a, __global float const *restrict b, __global float *restrict c, int const n)
{

  for (int j = 0; j <= -1 + n; ++j)
    for (int i = 0; i <= -1 + n; ++i)
      c[n * i + j] = a[i] * b[j];
}


/home/andreas/src/loopy/loopy/compiled.py:841: LoopyWarning: kernel scheduling was ambiguous--more than one schedule found, ignoring
  kernel = get_one_scheduled_kernel(kernel)


## Python glue for ease of use: The kernel "wrapper"

Invocation details handled by generated Python "wrapper":

* Argument derivation
* Data copying (from/to host, if requested)
* Shape/type/data layout checks

In [7]:
wknl = lp.set_options(knl, write_wrapper=True, write_cl=False)
evt, (mat,) = wknl(queue, a=a, b=b)

from __future__ import division

import pyopencl as _lpy_cl
import pyopencl.array as _lpy_cl_array
import pyopencl.tools as _lpy_cl_tools
import numpy as _lpy_np
from struct import pack as _lpy_pack

def invoke_loopy_kernel_loopy_kernel(cl_kernel, queue, allocator=None, wait_for=None, out_host=None, a=None, b=None, c=None, n=None):
    if allocator is None:
        allocator = _lpy_cl_tools.DeferredAllocator(queue.context)

    # {{{ find integer arguments from shapes

    if n is None:
        if a is not None:
            n = a.shape[0]
        elif b is not None:
            n = b.shape[0]
        elif c is not None:
            n = c.shape[0]
        elif c is not None:
            n = c.shape[1]

    # }}}

    # {{{ find integer arguments from offsets

    # }}}

    # {{{ find integer arguments from strides

    # }}}

    # {{{ process n

    if n is None:
        raise RuntimeError("input argument 'n' must "
            "be supplied")
    cl_kernel.set_arg(3, _lpy_pack('i', n)

/home/andreas/src/loopy/loopy/diagnostic.py:60: LoopyAdvisory: No device parameter was passed to the PyOpenCLTarget. Perhaps you want to pass a device to benefit from additional checking. (add 'no_device_in_pre_codegen_checks' to silenced_warnings kernel argument to disable)
  warn(text, type)


## Transforming kernels: Loop Splitting

Next: transform kernel. Example: Split a loop into fixed-length "chunks".

In [7]:
isplit_knl = knl
isplit_knl = lp.split_iname(isplit_knl, "i", 4)

evt, (mat,) = isplit_knl(queue, a=a, b=b)

#define int_floor_div_pos_b(a,b) (                 ( (a) - ( ((a)<0) ? ((b)-1) : 0 )  ) / (b)                 )
#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(__global float const *restrict a, __global float const *restrict b, __global float *restrict c, int const n)
{

  for (int i_outer = 0; i_outer <= -1 + int_floor_div_pos_b(3 + n, 4); ++i_outer)
    for (int j = 0; j <= -1 + n; ++j)
      for (int i_inner = 0; i_inner <= 3; ++i_inner)
        if (-1 + -1 * i_inner + -4 * i_outer + n >= 0)
          c[n * (i_inner + i_outer * 4) + j] = a[i_inner + i_outer * 4] * b[j];
}


Want to get rid of the conditional?

## Transforming kernels: Implementation Tags

Every loop axis ("iname") comes with an *implementation tag*.

In [8]:
isplit_knl = knl
isplit_knl = lp.assume(isplit_knl, "n mod 4 = 0")
isplit_knl = lp.split_iname(isplit_knl, "i", 4)
isplit_knl = lp.tag_inames(isplit_knl, {"i_inner": "unr"})

evt, (mat,) = isplit_knl(queue, a=a, b=b)

#define int_floor_div_pos_b(a,b) (                 ( (a) - ( ((a)<0) ? ((b)-1) : 0 )  ) / (b)                 )
#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(__global float const *restrict a, __global float const *restrict b, __global float *restrict c, int const n)
{

  for (int i_outer = 0; i_outer <= int_floor_div_pos_b(-4 + n, 4); ++i_outer)
    for (int j = 0; j <= -1 + n; ++j)
    {
      c[n * (0 + i_outer * 4) + j] = a[0 + i_outer * 4] * b[j];
      c[n * (1 + i_outer * 4) + j] = a[1 + i_outer * 4] * b[j];
      c[n * (2 + i_outer * 4) + j] = a[2 + i_outer * 4] * b[j];
      c[n * (3 + i_outer * 4) + j] = a[3 + i_outer * 4] * b[j];
    }
}


May want to influence loop ordering.

----
"Map to GPU hw axis" is an iname tag as well.

Use shortcuts for less typing:

In [9]:
split_knl = knl
split_knl = lp.split_iname(split_knl, "i", 16,
        outer_tag="g.0", inner_tag="l.0")
split_knl = lp.split_iname(split_knl, "j", 16,
        outer_tag="g.1", inner_tag="l.1")

evt, (mat,) = split_knl(queue, a=a, b=b)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(16, 16, 1))) loopy_kernel(__global float const *restrict a, __global float const *restrict b, __global float *restrict c, int const n)
{

  if (
      -1 + -16 * gid(1) + -1 * lid(1) + n >= 0
      && -1 + -16 * gid(0) + -1 * lid(0) + n >= 0
    )
    c[n * (lid(0) + gid(0) * 16) + lid(1) + gid(1) * 16] = a[lid(0) + gid(0) * 16] * b[lid(1) + gid(1) * 16];
}


/home/andreas/src/loopy/loopy/diagnostic.py:60: LoopyAdvisory: No device parameter was passed to the PyOpenCLTarget. Perhaps you want to pass a device to benefit from additional checking. (add 'no_device_in_pre_codegen_checks' to silenced_warnings kernel argument to disable)
  warn(text, type)


## Transforming kernels: Leveraging data reuse

Better! But still not much data reuse.

In [10]:
fetch1_knl = knl

fetch1_knl = lp.add_prefetch(fetch1_knl, "a")
fetch1_knl = lp.add_prefetch(fetch1_knl, "b")

evt, (mat,) = fetch1_knl(queue, a=a, b=b)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(__global float const *restrict a, __global float const *restrict b, __global float *restrict c, int const n)
{
  float b_fetch_0;
  float a_fetch_0;

  for (int i = 0; i <= -1 + n; ++i)
  {
    a_fetch_0 = a[i];
    for (int j = 0; j <= -1 + n; ++j)
    {
      b_fetch_0 = b[j];
      c[n * i + j] = a_fetch_0 * b_fetch_0;
    }
  }
}


But this is useless for the GPU version. (demo)

---

Would like to fetch entire "access footprint" of a loop.

In [11]:
fetch_knl = split_knl

fetch_knl = lp.add_prefetch(fetch_knl, "a", ["i_inner"])
fetch_knl = lp.add_prefetch(fetch_knl, "b", ["j_inner"])

evt, (mat,) = fetch_knl(queue, a=a, b=b)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(16, 16, 1))) loopy_kernel(__global float const *restrict a, __global float const *restrict b, __global float *restrict c, int const n)
{
  __local float b_fetch_0[16];
  __local float a_fetch_0[16];

  if (-1 + -16 * gid(0) + -1 * lid(0) + n >= 0)
    a_fetch_0[lid(0)] = a[lid(0) + 16 * gid(0)];
  if (-1 + -16 * gid(1) + -1 * lid(0) + n >= 0)
    b_fetch_0[lid(0)] = b[lid(0) + 16 * gid(1)];
  barrier(CLK_LOCAL_MEM_FENCE) /* for a_fetch_0 (insn depends on a_fetch) */;
  if (
      -1 + -16 * gid(1) + -1 * lid(1) + n >= 0
      && -1 + -16 * gid(0) + -1 * lid(0) + n >= 0
    )
    c[n * (lid(0) + gid(0) * 16) + lid(1) + gid(1) * 16] = a_fetch_0[lid(0)] * b_fetch_0[lid(1)];
}


## Transforming kernels: Eliminating Conditionals

All those conditionals take time to evaluate!

In [8]:
sfetch_knl = knl
sfetch_knl = lp.split_iname(sfetch_knl, "i", 16,
        outer_tag="g.0", inner_tag="l.0", slabs=(0,1))
sfetch_knl = lp.split_iname(sfetch_knl, "j", 16,
        outer_tag="g.1", inner_tag="l.1", slabs=(0,1))

sfetch_knl = lp.add_prefetch(sfetch_knl, "a", ["i_inner"])
sfetch_knl = lp.add_prefetch(sfetch_knl, "b", ["j_inner"])

evt, (mat,) = sfetch_knl(queue, a=a, b=b)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(16, 16, 1))) loopy_kernel(__global float const *restrict a, __global float const *restrict b, __global float *restrict c, int const n)
{
  __local float a_fetch[16];
  __local float b_fetch[16];

  /* bulk slab for 'j_outer' */

  /* bulk slab for 'i_outer' */

  if (
      -17 + -16 * gid(1) + n >= 0
      && -17 + -16 * gid(0) + n >= 0
    )
  {
    b_fetch[lid(0)] = b[lid(0) + 16 * gid(1)];
    a_fetch[lid(0)] = a[lid(0) + 16 * gid(0)];
    barrier(CLK_LOCAL_MEM_FENCE) /* for a_fetch (insn depends on a_fetch_rule) */;
    c[n * (lid(0) + gid(0) * 16) + lid(1) + gid(1) * 16] = a_fetch[lid(0)] * b_fetch[lid(1)];
  }
  /* final slab for 'i_outer' */

  if (
      16 + 16 * gid(0) + -1 * n >= 0
      && -17 + -16 * gid(1) + n >= 0
    )
  {
    b_fetch[lid(0)] = b[lid(0) + 16 * gid(1)];
    if (-1 + -16 * gid(0) + -1 * lid(0) + n >= 0)
      a_fetch[lid(0)] =